In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [2]:
covariances = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/Covariances.csv')
means = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/Means.csv')
trees = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/Trees.csv')

# QUESTION 1 - TREES

The 50 million tree program is a tree planting charity whose mandate is to increase forest cover in
Ontario. As of 2020, more than 30 million trees have been planted. This year, the organization made
a big push to acquire funding so that they could plant 10 million trees in 2022. There are a total of
18 potential planting locations in Ontario but it remains to determine which sites should be chosen.
There is a fee per location that is incurred if planting site i = 1, ..., 18 is chosen and there is a cost
associated with planting each tree (see the file Trees.csv for more information). Unfortunately, due
to the Conservation Authorities Act of Ontario, many intricate laws must be adhered to.
- Between 103,000 and 970,000 trees can be planted at any location if selected.
- At most two planting location can be chosen amongst the sites 1, 2, 3, and 4.
- Exactly three planting locations must be chosen amongst the sites 6, 9, 12, 15, and 18.
- No more than 4 planting locations must be chosen among sites 2, 4, 6, 8, 12, 14, 16, and 18.
- If planting location 5 is chosen then the sites 6, 7, and 8 cannot be chosen.
- If planting location 9 is chosen then at least two of sites 13, 15, and 17 must be chosen.
- The sum of all trees planted at sites 1-9 must equal the sum of all trees planted at sites 10-18.

Formulate a MILP model to minimize the sum of costs related to planting the 10 million trees while
respecting the legal requirements in the Conservation Act. Then, answer the following 10 questions.

A) Without solving the MILP, what is the minimum number of planting locations given the maximum number of trees that can be planted at any location?

B) Does the objective contain fixed costs only, variable costs only, or both?

C) Write down the constraints associated with linking the integer and binary decision variables.

D) If there were no legal requirements (the Conservation Authorities Act constraints), what location(s) would you choose to plant the trees in order to minimize cost?

E) Write down the constraints associated with planting locations 9, 13, 15, and 17.

F)How many decision variables are in the formulation?

G) What is the optimal planting cost?

H) How many planting locations are used?

I) How many trees are planted at location 1?

J) Would it be worth it to negotiate a contract such that the charity pays a fixed fee of $200,000 per location but could plant between 103,000 and 970,000 trees without any other costs?

# QUESTION 2 - PUFF DONUTS

# QUESTION 3 - MEANS

# QUESTION 4 - CRUISES